<a href="https://colab.research.google.com/github/henrique4d/CG---Trab3/blob/main/NB05_Minera%C3%A7%C3%A3o_de_Padr%C3%B5es_Frequentes_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Notebook 5. Mineração de Padrões Frequentes - Parte 01

<p>Notebook de apoio à Lição 2: "Mineração de Padrões Frequentes - Parte 1".</p>

<p>Objetivos:</p>

- Importar, processar e preparar dados para mineração de padrões frequentes
- Implementar o algoritmo ingênuo (básico) visto em aula

## 1. Importação de bibliotecas úteis

- seaborn e matplotlib: visualização de dados
- numpy: biblioteca numérica/matemática e de manipulação de arrays
- pandas: importação e manipulação de estruturas de dados (DataFrames)
- itertools: análise combinatória (permutações, combinações etc)
- pprint: impressão "amigável" de estruturas de dados complexas
- time: medição de tempo de execução
- mlxtend: aprendizado de máquina (que tem padrões frequentes)
- IPython.display.Latex: exibição de formatação LaTeX para notebooks

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import itertools as itt
import pprint as pp
from time import time
from mlxtend.frequent_patterns import apriori, association_rules
from IPython.display import Latex

## 2. Preparação da base de dados

- Os dados estão em um arquivo texto, no formato transacional
- Desta vez, vamos carregar os dados e manipular a informação "manualmente"
- Ao executar em seu computador, pode ser que seja necessário alterar o caminho e/ou nome do arquivo

### 2.1 Carga dos dados

In [ ]:
# Carrega conteúdo do arquivo para uma lista
fp = open('dados/padaria.dat', 'r')
# Carrega os dados do arquivo para uma lista de strings, onde cada item da lista é uma linha do arquivo
conteudo = fp.readlines()
fp.close()

### 2.2 Criação da representação transacional a partir do arquivo

In [ ]:
# Inicializa estruturas da base de dados e lista de itens
I = set()
base_t = []

# Varre conteúdo do arquivo
for linha in conteudo:
    # Elimina espaços em branco do início e final da linha
    linha = linha.strip()
    # Quebra linha usando a vírgula e espaço como separadores
    t = linha.split(', ')
    # Transforma o tid em inteiro
    t[0] = int(t[0])
    # Extrai itens
    itens = set(t[1:])
    # Inclui itens no conjunto I
    I = I.union(itens)
    
    # Adiciona a transação na base
    base_t.append(t)

# Ordena I
I = sorted(I)

# Exibe resultado
print(I)

### 2.3 Criação da representação binária a partir da representação transacional

In [ ]:
# Cria matriz booleana de zeros, e dimensões |base_t| x |I|
dados_bin = np.zeros((len(base_t), len(I)), dtype=bool)
# Inicializa lista de ids de transação
tids = []
# Percorre as linhas da base transacional
for t in base_t:
    # Extrai tid e acrescenta em tids
    tid = t[0]
    tids.append(tid)
    # i é a linha do array
    i = tid-1
    #    Percorre os itens da transação, identifica em qual coluna j estão
    # e define o dado da posição i,j como verdadeiro
    for item in t[1:]:
        j = I.index(item)
        dados_bin[i,j] = True
# Cria e exibe DataFrame
df = pd.DataFrame(data=dados_bin, columns=I, index=tids)
df

### 2.4 Salvamento da base binária

In [ ]:
df.to_pickle('dados/padaria.pkl')

## 3. Mineração de Padrões Frequentes

### 3.1 Definindo parâmetros

<p>O que seria algo frequente na base de dados?</p>

<p>Olhar a distribuição dos dados pode nos auxiliar nessa tarefa.</p>

#### 3.1.1 Análise exploratória

In [ ]:
probabilidades = []
for item in I:
    contagem = df[item].sum()
    probabilidades.append(contagem)
    
probabilidades = np.array(probabilidades)/len(df)
prob_media = np.mean(probabilidades)
desv_prob = np.std(probabilidades)
    
plt.title('Probabilidade de Ocorrência de cada item')
plt.bar(I, probabilidades, label='Densidade')
plt.axhline(y=prob_media, color='k', ls='dashed', label='Média')
plt.axhline(y=prob_media-desv_prob, color='orange', ls='dashed')
plt.axhline(y=prob_media+desv_prob, color='orange', ls='dashed', label='Desv. Pad.')
plt.grid()
plt.xticks(rotation='vertical')
plt.legend()
plt.tight_layout()
plt.show()

display(Latex(r'$\bar{x}$: %7.4f' % prob_media))
display(Latex(r'$\bar{x} - s$: %7.4f' % (prob_media - desv_prob)))
display(Latex(r'$\bar{x} + s$: %7.4f' % (prob_media + desv_prob)))

#### 3.1.2 Suporte e Confiança Mínimos

In [ ]:
# minconf: arbitrário (especialista)
minconf = 0.7
# minsup: estimado com análise exploratória e validado por especialista
minsup = prob_media - 0.5*desv_prob
print('Minsup:', minsup)

### 3.2 Enumeração de conjuntos frequentes

<p>Na implementação ingênua, utilizaremos um algoritmo de <b>força bruta</b>.</p>

In [ ]:
inicio = time()

# Inicialização: conjunto F e mapa de suportes
F = []
S = {}

# Enumeração e validação de conjuntos para cada tamanho de k
para = False
for k in range(1, len(I)+1):
    # Gera as combinações de tamanho k e modela para uma lista
    for itemset in itt.combinations(I, k):
        # Monta representação binária do itemset
        itemset_bin = np.zeros(len(I), dtype=bool)
        for item in itemset:
            pos = I.index(item)
            itemset_bin[pos] = True
        # Calcula suporte
        sup = 0
        for it in range(len(df)):
            t = df.iloc[it]
            check = t & itemset_bin
            if (check == itemset_bin).all():
                sup += 1
        sup /= len(df)
                
        # Cria entrada no mapa de suportes
        S[itemset] = sup
                
        # Valida frequencia
        if sup >= minsup:
            F.append(itemset)
            
tempo_f = time() - inicio

print('Conjuntos obtidos:\n')
pp.pprint(F)
print()
print('Em %7.4fs' % tempo_f)

### 3.3 Geração e validação de regras

In [ ]:
inicio = time()

# Inicializa conjunto final de regras
R = []
# Para cada permutação em F de tamanho 2:
for r in itt.permutations(F, 2):
    # Extrai o antecedente (X) e o consequente (Y)
    X = set(r[0])
    Y = set(r[1])
    # Considerar somente os casos onde X e Y não possuem interseção
    if X.intersection(Y) == set():
        # Gera um conjunto (ordenado) XY (união)
        XY = tuple(sorted(X.union(Y)))
        # Obtem suporte de XY
        sup = S[XY]
        # Valida suporte
        if sup >= minsup:
            # Calcula, valida confiança e acrescenta em R, se for o caso
            conf = sup / S[tuple(sorted(X))]
            if conf >= minconf:
                R.append((r, sup, conf))
                
tempo_r = time() - inicio

print('Regras obtidas:\n')
pp.pprint(R)
print()
print('Em %7.4fs' % tempo_r)

### 3.4 Validando resultados com a biblioteca <code>mlxtend</code>

<p>O que seria algo frequente na base de dados?</p>

<p>Olhar a distribuição dos dados pode nos auxiliar nessa tarefa.</p>

In [ ]:
inicio = time()

frequent_itemsets = apriori(df, min_support=minsup, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=minconf)

tempo_mlx = time() - inicio

print('Regras obtidas:')
display(rules[['antecedents', 'consequents', 'support', 'confidence']])
print('Em %7.4fs' % tempo_mlx)